### 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용: 뉴스 키워드 추출기

In [90]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate, 
    ChatPromptTemplate
)
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [91]:
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news":
            """
            제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다.
            """,
        "keywords":"제미나이, 구글AI, 개발자"
    },
    {
        "news": "삼성전자가 AI 가속기를 출시할 예정이다.",
        "keywords": "삼성전자, AI가속기, 출시"
    },
    {
        "news": "WHO가 국제 협력의 중요성을 강조했다.",
        "keywords": "WHO, 국제협력, 보건"
    }
]


In [92]:
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.0
)

In [93]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])


In [94]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples = examples
)

In [95]:
final_prompt = ChatPromptTemplate.from_messages(
    [("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
    few_shot_prompt,
    ("human", "{news_test}")]
)

In [96]:
chain = final_prompt | llm | StrOutputParser()

In [97]:
test_news = """ 
    이재명 대통령이 토큰증권(STO) 법제화를 공약으로 내놓은 가운데 금융투자업계에선 시장 생태계를 선점하려는 움직임이 시작됐다.
11일 금융투자업계에 따르면 주요 증권사들은 이미 STO 관련 시스템 구축을 마치고 제도 인가를 기다리는 상황이다. 지난해부터 STO 관련 법안의 국회 통과가 미뤄지고 번번이 무산되면서 투자업계는 불확실성 속에 힘이 빠진 상태였다.

이 대통령이 이끄는 새 정부가 출범하면서 STO 법제화 시기가 7~8월로 전망된다. 이에 투자업계는 다시 관련 사업에 박차를 가하고 있다. 여야 모두 STO 법제화의 필요성에 공감해 통과에 무리가 없을 것이라는 전망이다.


증권사들은 금융사와 함께 STO 협의체를 맺고 있다. 미래에셋증권은 하나금융그룹, SK텔레콤과 함께 '넥스트파이낸스이니셔티브(NFI)'를 구성하고, 자체 STO 메인넷 개발을 완료했다. 삼성증권도 우리은행, SK증권과 STO 협의체 'F3P(Finance 3.0 Partners)'를 구성했다. KB증권은 신한투자증권, NH투자증권과 함께 '토큰증권 컨소시엄'을 꾸리고 공동 인프라를 구축하고 있다. 신한투자증권은 SK증권, 블록체인글로벌과 함께 '펄스(PULSE)' 프로젝트를 추진 중이다.

NH투자증권은 농협은행, 케이뱅크, 펀블 등과 함께 'STO 비전그룹'을 결성하고 공동 생태계 설계를 추진하고 있다. 금융 인프라와 디지털 기술 기업이 참여하는 다자 협력 모델을 통해 실물 기반 자산의 구조화와 유통을 단계적으로 확대할 계획이다. 한국투자증권은 카카오뱅크, 토스뱅크와 '한국투자ST프렌즈' 협의체를 구성했다.

금융투자업계 관계자는 "토큰증권은 동일한 증권에 대해 하나의 증권사가 발행과 유통을 겸업할 수 없는 구조"라며 "발행·유통 협업 구조를 타 증권사와 만들어 나갈 수 있다"고 설명했다. 이어 "금융위원회 분산원장 요건에 3개 이상의 금융회사가 노드에 참여하는 것을 명시해놨으므로 노드 다양화 측면에서도 시너지가 날 것"이라고 덧붙였다. 노드는 분산 원장의 핵심 구성 요소로서 블록체인이 안전하고 효율적으로 작동하도록 돕는 컴퓨터나 장치를 뜻한다.

━
은행권·STO 플랫폼 등 STO 사업 박차

━

사진은 기사 내용과 관련 없음. /사진=클립아트코리아

사진은 기사 내용과 관련 없음. /사진=클립아트코리아
증권사는 금융사뿐만 아니라 STO 발행사들과도 업무협약을 체결하고 있다. 하나증권은 STO 기반 디지털자산운용 플랫폼 '피스'를 운영하는 바이셀스탠다드와 지난달 업무협약을 체결했고, LS증권도 신재생에너지 기반 STO 사업을 위해 하이카이브와 파트너십을 맺었다. KB증권은 STO 발행·유통 사업자 협력체들과 'ST 오너스'를 구성했다. 주요 사업자로는 서울옥션블루, 펀더풀, 하이카이브, 오아시스비즈니스 등이 22개사가 참여한다.
바이셀스탠다드는 2019년 설립됐는데 신탁형과 투자계약형 증권을 모두 취급할 수 있는 역량을 보유하고 있는 국내 유일 플랫폼으로 평가받는다. 희소성 높은 현물·미술품·선박·지식재산권(IP) 등 폭넓은 실물자산의 토큰화가 가능한 시스템을 구축했다.


은행권에서는 NH농협은행을 중심으로 신한·우리·국민·기업·전북·수협은행이 공동으로 참여하는 '은행권 STO 컨소시엄'이 출범한 상태다. 디지털 자산 유통 채널을 은행권과 연결하려는 시도들이 이어지고 있다.

STO 플랫폼들은 실물자산의 디지털 전환을 담당하는 핵심 축으로서 ▲자산 감정 ▲토큰 설계 ▲스마트계약 구현 ▲투자자 유치 등 종합적인 서비스를 제공해 전통 자본시장과 블록체인 기술을 잇는 가교 역할을 수행한다.

부동산 중심 플랫폼에는 카사, 펀블, 루센트블록, 비브릭 등이 있다. 카사는 압구정 상업용 빌딩 공모를 통해 167억원 모집에 성공하며 대규모 공모형 STO의 실현 가능성을 보여줬다. 펀블은 KOTRA 글로벌 거점사업 선정을 계기로 두바이 중심의 중동 진출을 본격화하고 있다. 케이카우는 친환경 한우농가와의 제휴를 통해 ESG(환경·사회·지배구조) 조각투자 모델을 개발해 최근 8차례 연속 완판 기록을 세웠다. 예스24 계열사 아피티오는 알렉스 카츠 등 유명 작가의 작품을 활용한 예술품 조각투자로 틈새시장을 공략하고 있다.

━
STO 법제화, 한국의 글로벌 디지털 금융 허브 도약 기회

━

사진은 기사 내용과 관련 없음. /사진=클립아트코리아

사진은 기사 내용과 관련 없음. /사진=클립아트코리아
STO 생태계의 기반 시설 구축도 빠르게 진행되고 있다. 한국예탁결제원은 토큰증권의 보관과 권리 변동 관리 업무를 맡게 되며, 한국거래소는 내년 디지털증권 전용 거래 시장 개설을 목표로 관련 시스템을 준비 중이다. 이들 기관은 ▲토큰 발행량 관리 ▲소유권 이전 처리 ▲결제 시스템 운영 등 STO 시장의 핵심 기능을 담당하며 투명하고 안정적인 거래 환경 조성에 핵심적 역할을 할 예정이다.
STO 법제화 기대효과를 살펴보면, 투자자는 신종자산이 토큰증권화돼 상품 선택지가 확대될 수 있다. 증권사 입장에선 블록체인을 활용해 증권을 발행하는 데 드는 비용과 시간이 줄어든다. 또, 주요 금융 선진국들은 블록체인 네트워크를 연결해 자국 금융 시장의 세계화를 추진 중인 가운데 국내 역시 이러한 움직임에 동참할 수 있게 된다.

STO 법제화는 한국이 글로벌 디지털 금융 허브로 도약할 절호의 기회라고 시장은 기대하고 있다. 글로벌 컨설팅 회사인 보스턴컨설팅그룹의 보고서에 따르면 전세계 토큰화 시장이 2030년까지 16조달러(약 2경1872조원) 규모로 성장할 것이라고 내다봤다. 2030년 국내 STO 시장은 367조원 규모로 전망했다.
    """

In [98]:
result = chain.invoke({"news_test":test_news})

In [99]:
print(result)

키워드: 토큰증권, 금융투자업계, 블록체인
